In [1]:
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

import torch
import sklearn
import polars as pl
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import Gemma2ForSequenceClassification, GemmaTokenizerFast, BitsAndBytesConfig
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import PeftModel
from peft import LoraConfig, get_peft_model

In [2]:
@dataclass
class Config:
    gemma_dir = '/root/autodl-tmp/WSDM/working/gemma-2-9b-it-bnb-4bit'
    lora_dir = '/root/autodl-tmp/WSDM/project/exp/all_data/best_last_epoch_0/adapter.bin'
    max_prompt_length: int = 512
    max_length = 2048
    batch_size = 8
    tta = False
cfg = Config()

In [3]:
tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)
tokenizer.add_eos_token = True
tokenizer.padding_side = "right"

In [22]:
valid_df = pl.read_parquet('/root/autodl-tmp/WSDM/input/hf-open-models-v2.parquet').to_pandas()

valid_df['response_b'] = valid_df.apply(
    lambda row: '\n\n<response_b>: ' + (str(row['response_b']) if pd.notnull(row['response_b']) else 'N/A'),
    axis=1
)

valid_df['response_a'] = valid_df.apply(
    lambda row: '\n\n<response_a>: ' + (str(row['response_a']) if pd.notnull(row['response_a']) else 'N/A'),
    axis=1
)

def process2(row, tokenizer):
    for col in ['prompt', 'response_a', 'response_b']:
        row[col] = row[col].fillna('')
        text_list = []
        if col == 'prompt':
            max_no = 512
            s_no = 256
            e_no = -256
        else:
            max_no = 1024
            s_no = 512
            e_no = -512
        for text in tqdm(row[col]):
            encoded = tokenizer(text, return_offsets_mapping=True)
            if len(encoded['input_ids']) > max_no:
                start_idx, end_idx = encoded['offset_mapping'][s_no]
                new_text = text[:end_idx]
                # print(len(tokenizer(text[:end_idx])['input_ids']))
                start_idx, end_idx = encoded['offset_mapping'][e_no]
                # print(len(tokenizer(text[start_idx:])['input_ids']))
                new_text = new_text + "\n(snip)\n" + text[start_idx:]
                # print(len(tokenizer(new_text)['input_ids']), new_text)
                text = new_text
            text_list.append(text)
        row[col] = text_list
    return row

valid_df =  process2(valid_df, tokenizer)

100%|██████████| 13093/13093 [00:10<00:00, 1245.36it/s]


In [5]:
def one_data_process(row):
    prompt = '<prompt>: ' + row['prompt']
    response_a = '\n\n<response_a>: ' + row['response_a']
    response_b = '\n\n<response_b>: ' + row['response_b']
    p = tokenizer(prompt, add_special_tokens=False)['input_ids']
    a = tokenizer(response_a, add_special_tokens=False)['input_ids']
    b = tokenizer(response_b, add_special_tokens=False)['input_ids']

    if len(p) > cfg.max_prompt_length:
        p = p[-cfg.max_prompt_length:]
    reponse_length = (cfg.max_length - len(p)) // 2  ##-2
    input_ids = \
        [tokenizer.bos_token_id] + \
        p + a[-reponse_length:] + b[-reponse_length:] + \
        [tokenizer.eos_token_id]
    length = len(input_ids)
    attention_mask = [1] * length
    return input_ids, attention_mask, length


valid_df[['input_ids', 'attention_mask', 'length']] = \
    valid_df.apply(one_data_process, axis=1, result_type='expand')
valid_df = valid_df.sort_values('length', ascending=False).reset_index(drop=True)
multi_df = [
    valid_df[0::2].reset_index(drop=True),
    valid_df[1::2].reset_index(drop=True),
]
valid_df = pd.concat(multi_df).reset_index(drop=True)

print('data ok!!!')

data ok!!!


In [6]:
valid_df

,id,prompt,response_a,response_b,model_a,model_b,language,__index_level_0__,input_ids,attention_mask,length
0,6c6de1fd9ff94dd5881902211240e32d,"Como puedo usar spring boot security, para el ...","\n\n<response_a>: Claro, te mostraré un ejempl...",\n\n<response_b>: Para usar Spring Boot Securi...,Qwen/Qwen2.5-72B-Instruct,microsoft/Phi-3-mini-4k-instruct,Spanish,8060,"[2, 235322, 39038, 78880, 21503, 39952, 18618,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2050
1,d095a2e62b6242a0871aae5108848517,Please make a research and suggest the list of...,\n\n<response_a>: Research into natural substa...,\n\n<response_b>: Here's a list of potential h...,01-ai/Yi-1.5-34B-Chat,meta-llama/Llama-3.3-70B-Instruct,English,2541,"[2, 235322, 39038, 78880, 5651, 1501, 476, 367...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2050
2,b2f84c2b9caf4188b6a9e77db350a470,Python code to solve sudoku,\n\n<response_a>: ```python\ndef is_valid(grid...,\n\n<response_b>: ```python\ndef is_valid(grid...,google/gemma-2-2b-it,google/gemma-2-9b-it,Latin,4143,"[2, 235322, 39038, 78880, 21237, 3409, 577, 11...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2050
3,587eeeffb53e4d52bd4edea8cda9b9de,Write an article about the Upstream and Downst...,\n\n<response_a>: **Title:** Exploring the Ups...,\n\n<response_b>: 3-Amino-4-methoxypyridine (...,meta-llama/Meta-Llama-3-8B-Instruct,mistralai/Mixtral-8x7B-Instruct-v0.1,English,9147,"[2, 235322, 39038, 78880, 15615, 671, 5078, 11...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2050
4,5d74e5ce1c2c46ef8a1bcb25e8fc01c6,Write an article about the Instruction of THIO...,\n\n<response_a>: ### Introduction to Thiophen...,\n\n<response_b>: **Title:** The Instruction o...,Qwen/Qwen2.5-72B-Instruct,meta-llama/Meta-Llama-3-8B-Instruct,English,8852,"[2, 235322, 39038, 78880, 15615, 671, 5078, 11...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2050
...,...,...,...,...,...,...,...,...,...,...,...
13088,59dc8d42dc1249bda07ceb02fa04aea6,主播你好,\n\n<response_a>: 大胆主播！,\n\n<response_b>: 嗨！有什么可以帮到你的吗？,meta-llama/Llama-3.2-1B-Instruct,mistralai/Mistral-Nemo-Instruct-2407,Chinese,9072,"[2, 235322, 39038, 78880, 31907, 237021, 87139...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",48
13089,fe2249080a764c558893a3bbc12d97bc,2×6=,\n\n<response_a>: 2 × 6 = 12,\n\n<response_b>: 12,meta-llama/Llama-3.3-70B-Instruct,microsoft/Phi-3-mini-4k-instruct,unknown,97,"[2, 235322, 39038, 78880, 235248, 235284, 2361...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",48
13090,ab1b2d76c4cb46eaba1e0b14b279e649,napisz cześć,\n\n<response_a>: Cześć!,\n\n<response_b>: Cześć! Jak mogę Ci pomóc dzi...,meta-llama/Llama-3.2-3B-Instruct,microsoft/phi-4,Polish,4572,"[2, 235322, 39038, 78880, 12987, 31708, 91285,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",48
13091,3091d4bb93b54597bfd60954593501c5,olá!,\n\n<response_a>: Olá! 😊 How are you?,\n\n<response_b>: Olá!,meta-llama/Llama-2-7b-chat-hf,meta-llama/Meta-Llama-3-8B-Instruct,Portuguese,11390,"[2, 235322, 39038, 78880, 6989, 235354, 235341...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",45


In [7]:
def one_data_process_aug(row):
    prompt = '<prompt>: ' + row['prompt']
    response_a = '\n\n<response_a>: ' + row['response_b']
    response_b = '\n\n<response_b>: ' + row['response_a']
    p = tokenizer(prompt, add_special_tokens=False)['input_ids']
    a = tokenizer(response_a, add_special_tokens=False)['input_ids']
    b = tokenizer(response_b, add_special_tokens=False)['input_ids']

    if len(p) > cfg.max_prompt_length:
        p = p[-cfg.max_prompt_length:]
    reponse_length = (cfg.max_length - len(p)) // 2  ##-2
    input_ids = \
        [tokenizer.bos_token_id] + \
        p + a[-reponse_length:] + b[-reponse_length:] + \
        [tokenizer.eos_token_id]
    length = len(input_ids)
    attention_mask = [1] * length
    return input_ids, attention_mask, length


In [8]:
# torch.cuda.empty_cache()
# del model

In [9]:
# Load base model on GPU 0
def load_model(device):
    # device_0 = torch.device('cuda:0')
    model = Gemma2ForSequenceClassification.from_pretrained(
        cfg.gemma_dir,
        use_cache=False,
        device_map=device,
        torch_dtype=torch.float16
    )
    model.score = torch.nn.Linear(in_features=3584, out_features=2, bias=False).to(device)
    # else:
    #     model = AutoModelForCausalLM.from_pretrained(args.model_name_or_path)
    # model = prepare_model_for_kbit_training(model)
    
    lora_config = LoraConfig(
                            r=256, 
                            lora_alpha=128, 
                            target_modules=[
                                    "q_proj",
                                    "k_proj",
                                    "v_proj",
                                    "o_proj",
                                    "gate_proj",
                                    "up_proj",
                                    "down_proj",
                                ],
                            bias="none",
                            lora_dropout=0.05, 
                            task_type="CAUSAL_LM"
                            )
    model = get_peft_model(model, lora_config)
    d = torch.load(cfg.lora_dir, map_location=model.device)
    # print(d)
    state_dic = {}
    for k,v in d.items():
        state_dic['base_model.model.' + k] = v
    # print(state_dic)
    # for k,v in state_dic.items():
    #     print(k)
    model.load_state_dict(state_dic, strict=False)
    model.to(device)
    model.eval()
    print('load ok!')
    return model

model_0 = load_model('cuda:0')
model_1 = load_model('cuda:1')
multi_model = [
    model_0,
    model_1
]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/WSDM/working/gemma-2-9b-it-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_98893/1417842169.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unp

load ok!


Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/WSDM/working/gemma-2-9b-it-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


load ok!


In [10]:
## inference #####################################################################
import torch.nn.functional as F
from tqdm import tqdm

def make_submission(valid_df, probability):
    # id,winner
    predict = probability.argmax(-1)
    winner =[
        {0:'model_a',1:'model_b',}[p] for p in predict
    ]
    submit_df = pd.DataFrame({
        'id' : valid_df['id'],
        'winner': winner,
    })
    return submit_df

def make_submission_prob(valid_df, probability):
    # id,winner
    predict = probability
    # winner =[
    #     {0:'model_a',1:'model_b',}[p] for p in predict
    # ]
    submit_df = pd.DataFrame({
        'id' : valid_df['id'],
        'model_a_prob': predict[:, 0],
        'model_b_prob': predict[:, 1],
    })
    return submit_df
    
def do_infer(model, valid_df, name=''):
    num_valid = len(valid_df)

    probability = []
    for i in tqdm(range(0, num_valid, cfg.batch_size), total=len(list(range(0, num_valid, cfg.batch_size)))):

        B = min(i +  cfg.batch_size, num_valid) - i
        d = valid_df[i:i + B]
        batch = {
            'input_ids': d['input_ids'].tolist(),
            'attention_mask': d['attention_mask'].tolist(),
        }
        batch = pad_without_fast_tokenizer_warning(
            tokenizer,
            batch,  #{"input_ids": input_ids, "attention_mask": attention_mask},
            padding='longest',
            pad_to_multiple_of=None,
            return_tensors='pt',
        )
        with torch.amp.autocast('cuda', enabled=True):
            with torch.no_grad():
                output = model(
                    input_ids=batch['input_ids'].to(model.device),
                    attention_mask=batch['attention_mask'].to(model.device),
                )
                p = F.softmax(output.logits, dim=1)
                probability.append(p.data.cpu().numpy())
                #print(probability.shape)
                #todo tta
    
    torch.cuda.empty_cache()
    print('')
    probability = np.concatenate(probability)
    return probability


if 0:
    #debug
    probability = do_infer( multi_model[0] , multi_df[0], name='')
    print(probability)
    # exit(0)


# start_timer = timer()
with ThreadPoolExecutor(max_workers=2) as executor:
    result = executor.map(do_infer, multi_model , multi_df, ('process0', 'process1'))
probability = np.concatenate(list(result))
# time_taken = timer() - start_timer



100%|██████████| 819/819 [20:42<00:00,  1.52s/it] 


100%|██████████| 819/819 [20:58<00:00,  1.54s/it]

In [11]:
if cfg.tta:
    aug_valid_df = valid_df.copy()
    aug_valid_df[['input_ids', 'attention_mask', 'length']] = \
        aug_valid_df.apply(one_data_process_aug, axis=1, result_type='expand')
    aug_valid_df = aug_valid_df.sort_values('length', ascending=False).reset_index(drop=True)
    multi_df_2 = [
        aug_valid_df[0::2].reset_index(drop=True),
        aug_valid_df[1::2].reset_index(drop=True),
    ]
    valid_df = pd.concat(multi_df_2).reset_index(drop=True)
    
    print('data ok!!!')

    with ThreadPoolExecutor(max_workers=2) as executor:
        result = executor.map(do_infer, multi_model , multi_df_2, ('process0', 'process1'))
    probability2 = np.concatenate(list(result))[...,::-1]
    probability = (probability2 + probability) / 2
    

In [14]:
submit_df = make_submission(valid_df, probability)
submit_df = make_submission_prob(valid_df, probability)
submit_df.to_csv('submission.csv', index=False)
#np.save('probability.npy', probability)

print(submit_df)
print(probability.shape)
# print(f'time for 10_000 (max  4.5 hr): {10_000/num_valid*time_taken/60/60:4.1f}')
# print(f'time for 25_000 (max 12.0 hr): {25_000/num_valid*time_taken/60/60:4.1f}')
# print('MODE', MODE)
print('submit ok!!!')

                                     id  model_a_prob  model_b_prob
0      6c6de1fd9ff94dd5881902211240e32d      0.690533      0.309467
1      d095a2e62b6242a0871aae5108848517      0.465479      0.534521
2      b2f84c2b9caf4188b6a9e77db350a470      0.470524      0.529476
3      587eeeffb53e4d52bd4edea8cda9b9de      0.421701      0.578299
4      5d74e5ce1c2c46ef8a1bcb25e8fc01c6      0.577525      0.422475
...                                 ...           ...           ...
13088  59dc8d42dc1249bda07ceb02fa04aea6      0.232093      0.767908
13089  fe2249080a764c558893a3bbc12d97bc      0.456046      0.543954
13090  ab1b2d76c4cb46eaba1e0b14b279e649      0.350852      0.649148
13091  3091d4bb93b54597bfd60954593501c5      0.284179      0.715821
13092  358e87f89a424c16ade13ba8a8764db4      0.374476      0.625524

[13093 rows x 3 columns]
(13093, 2)
submit ok!!!


In [15]:
probability

array([[0.690533  , 0.30946702],
       [0.46547863, 0.5345214 ],
       [0.47052372, 0.5294763 ],
       ...,
       [0.3508522 , 0.6491478 ],
       [0.28417853, 0.71582144],
       [0.37447613, 0.62552387]], dtype=float32)

In [23]:
valid_df = pd.merge(valid_df, submit_df, on='id')
valid_df.shape

(13093, 10)

In [25]:
valid_df.to_csv('submission.csv', index=False)

In [26]:
import numpy as np
filter = valid_df[(valid_df['model_a_prob'] > 0.55) | (valid_df['model_b_prob'] > 0.55)]
filter['winner'] = np.where(filter['model_a_prob'] > filter['model_b_prob'], 'model_a', 'model_b')
filter[['id', 'prompt',	'response_a','response_b','winner',	'model_a',	'model_b',	'language']].to_csv('/root/autodl-tmp/WSDM/input/extra.csv', index=False)

/tmp/ipykernel_98893/1879665460.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter['winner'] = np.where(filter['model_a_prob'] > filter['model_b_prob'], 'model_a', 'model_b')


In [28]:
filter.value_counts('winner')

winner
model_b    5899
model_a    5667
Name: count, dtype: int64